# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import os, sys
from importlib import reload
from config import data_config
from utils.args_parser import add_data_args

# Reload arg_parser because Jupyter notebook does not reload it automatically
reload(data_config)

# Add arguments to parser
parser = ArgumentParser()
parser = add_data_args(parser)

# Print default dataset parameters
args = parser.parse_args([])
print("\n=====================")
print("Default dataset parameters:")
for arg in vars(args):
    print(arg, '=', getattr(args, arg))
print("\n=====================")




Default dataset parameters:
ode_system = pendulum
t_max = 20
step_size = 0.01
n_sample = 10
x_init_pts = [[-3.14159265  3.14159265]
 [-8.          8.        ]]
ctr_func = gaussian
datafile_path = data/pendulum_ctr_gaussian_N_10_T20.npy



In [2]:
# Run database_generator.py with arguments
print("\n=====================")
print("Running database_generator.py with arguments:")
%run database_generator.py \
    --ode_system "pendulum" \
    --step_size 0.01 \
    --n_sample 100 \
    --ctr_func "gaussian" \
    --t_max 10 \
    --datafile_path "data/pendulum_ctr_grf_N_100_h001_T10_2.npy"



Running database_generator.py with arguments:


Generating 100 data ...:   4%|▍         | 4/100 [00:00<00:19,  5.02it/s]/PENDULUM/src/config/data_config.py:35: RuntimeWarning: invalid value encountered in subtract
  u - b * omega - 0.5 * m * l * g * np.sin(theta)
/PENDULUM/src/config/data_config.py:35: RuntimeWarning: invalid value encountered in sin
  u - b * omega - 0.5 * m * l * g * np.sin(theta)
Generating 100 data ...: 100%|██████████| 100/100 [00:21<00:00,  4.61it/s]


Data saved in data/pendulum_ctr_grf_N_100_h001_T10_2.npy .


In [2]:
%run database_generator.py --help

usage: database_generator.py [-h] [--ode_system {pendulum,lorentz}]
                             [--t_max T_MAX] [--step_size STEP_SIZE]
                             [--n_sample N_SAMPLE]
                             [--x_init_pts [X_INIT_PTS ...]]
                             [--ctr_func {designate,gaussian}]
                             [--datafile_path DATAFILE_PATH]

options:
  -h, --help            show this help message and exit
  --ode_system {pendulum,lorentz}
                        ODE system to use.
  --t_max T_MAX         Maximum time (seconds) of the trajectory.
  --step_size STEP_SIZE
                        Step size (seconds) of the trajectory.
  --n_sample N_SAMPLE   Number of the trajectories to generate.
  --x_init_pts [X_INIT_PTS ...]
                        The range of initial points of the trajectories. Two
                        floats define the range of each state component.
  --ctr_func {designate,gaussian}
                        Control function to use.
  

In [17]:
import pandas as pd
import numpy as np

datafile_path = "/PENDULUM/archive/data/pendulum_u_random_init_a_001.pkl"
data = pd.read_pickle(datafile_path)
print(type(data["u"]))
print(data["u"].shape)
print(data["theta"].shape)
print(data["omega"].shape)
print(data["t"].shape)
data_new = dict()
data_new["u"] = np.expand_dims(data["u"].T, axis=1)
data_new["x"] = np.dstack((data["theta"].T, data["omega"].T)).transpose(0, 2, 1)
data_new["t"] = data["t"]
print(data_new["u"].shape)
print(data_new["x"].shape)
idxs_all = np.arange(data_new["u"].shape[2])
idxs_nan_u = np.isnan(data_new["u"]).sum(axis=(0,1)) 
idxs_nan_x = np.isnan(data_new["x"]).sum(axis=(0,1))
idxs_nan = idxs_nan_u + idxs_nan_x
idxs_valid = idxs_all[idxs_nan==0]
print(idxs_all.shape)
print(idxs_valid.shape)
data_new["u"] = data_new["u"][:, :, idxs_valid]
data_new["x"] = data_new["x"][:, :, idxs_valid]
np.save("/PENDULUM/src/data/pendulum_ctr_grf_N_5000_T10_2.npy", data_new)

<class 'numpy.ndarray'>
(5000, 999)
(5000, 999)
(5000, 999)
(999, 1)
(999, 1, 5000)
(999, 2, 5000)
(5000,)
(4820,)


In [19]:
import pandas as pd
import numpy as np

datafile_path = "/PENDULUM/archive/data/pendulum_u_test_random_init_a_001_stat.pkl"
data = pd.read_pickle(datafile_path)
print(type(data["u"]))
print(data["u"].shape)
print(data["theta"].shape)
print(data["omega"].shape)
print(data["t"].shape)
data_new = dict()
data_new["u"] = np.expand_dims(data["u"].T, axis=1)
data_new["x"] = np.dstack((data["theta"].T, data["omega"].T)).transpose(0, 2, 1)
data_new["t"] = data["t"]
print(data_new["u"].shape)
print(data_new["x"].shape)
idxs_all = np.arange(data_new["u"].shape[2])
idxs_nan_u = np.isnan(data_new["u"]).sum(axis=(0,1)) 
idxs_nan_x = np.isnan(data_new["x"]).sum(axis=(0,1))
idxs_nan = idxs_nan_u + idxs_nan_x
idxs_valid = idxs_all[idxs_nan==0]
print(idxs_all.shape)
print(idxs_valid.shape)
data_new["u"] = data_new["u"][:, :, idxs_valid]
data_new["x"] = data_new["x"][:, :, idxs_valid]
np.save("/PENDULUM/src/data/pendulum_ctr_grf_N_100_T10_2.npy", data_new)


<class 'numpy.ndarray'>
(120, 999)
(120, 999)
(120, 999)
(999, 1)
(999, 1, 120)
(999, 2, 120)
(120,)
(115,)


In [1]:
import pandas as pd
import numpy as np
from utils.nn_lib import Customize_Dataset
import torch
from torch.utils.data import DataLoader, random_split
from utils.data_utils import split_dataset, prepare_local_predict_dataset, scale_and_to_tensor, Dataset_Stat, Dataset_Torch
from config import train_config

seed = 999
np.random.seed(seed)
torch.manual_seed(seed)

datafile_path = "/PENDULUM/archive/data/pendulum_u_test_random_init_a_001_stat.pkl"
config = dict()
config.update(train_config.get_config())

dataset_o = Customize_Dataset(
        filepath=datafile_path,
        search_len=10,
        search_num=20,
        use_padding=True,
        search_random = True,
        device=torch.device("cpu"),
        transform=None,
        target_transform=None,
    )

print(dataset_o.metadata[0:20,-2])

datafile_path = "/PENDULUM/src/data/pendulum_ctr_grf_N_100_T10_2.npy"
dataset = np.load(datafile_path, allow_pickle=True).item()
data, _ = split_dataset(
        dataset, test_size=0., verbose=False
    )
input_masked, x_n, x_next, t_params = prepare_local_predict_dataset(
        data=data,
        state_component=config["state_component"],
        t_max=config["t_max"],
        search_len=config["search_len"],
        search_num=config["search_num"],
        search_random=config["search_random"],
        offset=config["offset"],
        verbose=config["verbose"],
    )
data_stat = Dataset_Stat(input_masked, x_n, x_next, t_params)
input_masked, x_n, x_next, t_params = scale_and_to_tensor(
        data_stat, scale_mode=config["scale_mode"]
    )
data_torch = Dataset_Torch(input_masked, x_n, x_next, t_params)
print(data_torch.x_next[0:20,0])

Data shape: torch.Size([2300, 999])
Data memory: 8.82 MB
tensor([-56.0552,   0.7273,  -0.7407,  -2.2929, -11.3510,   1.6141,  12.0482,
         -0.8410,  -1.5140,   1.2153,  -0.6183,  23.9170,  -0.1437,   0.6723,
        -41.4200,  -0.4753, -47.2333,  -2.3165,  -0.2414,   1.7439])
Shapes for LSTM-MIONet training input=(2300, 999, 1), x_n=(2300, 1), x_next=(2300, 1)
Data memory size: 8 MB
tensor([-56.0552,   0.7273,  -0.7407,  -2.2929, -11.3510,   1.6141,  12.0482,
         -0.8410,  -1.5140,   1.2153,  -0.6183,  23.9170,  -0.1437,   0.6723,
        -41.4200,  -0.4753, -47.2333,  -2.3165,  -0.2414,   1.7439])


In [3]:
# Run database_generator.py with arguments
import numpy as np

print("\n=====================")
print("Running database_generator.py with arguments:")
%run database_generator.py \
    --ode_system "lorentz" \
    --step_size 0.01 \
    --n_sample 100 \
    --t_max 20 \
    --x_init_pts -17 20 -23 28 0 50 \
    --datafile_path "data/lorentz_N_100_h001_T20.npy"


Running database_generator.py with arguments:


Generating 100 data ...: 100%|██████████| 100/100 [00:26<00:00,  3.73it/s]

Data saved in data/lorentz_N_100_h001_T20.npy .


In [13]:
import pandas as pd
# Read txt file
datafile_path_pred = "../archive/data/pendulum_infer_theta_single_chris_osc_pred.txt"
y_pred = pd.read_csv(datafile_path_pred, sep=" ")
datafile_path_true = "../archive/data/pendulum_infer_theta_single_chris_osc_true.txt"
y_true = pd.read_csv(datafile_path_true, sep=" ")
print(y_pred.shape)
print(y_true.shape)
L2_error = np.linalg.norm(y_pred.iloc[:,1] - y_true.iloc[:,1], axis=0) / np.linalg.norm(y_true.iloc[:,1], axis=0)
print(L2_error*100)

(998, 2)
(998, 2)
3.30081828012151
